# 데이터 재가공

### 질병 진단 결과에 따른 영양제 라벨링
- **당뇨** : metformine- 또는 오메가3
- **간** :  우루사
- **빈혈** : 철분제
- **비만** :  시서스 또는 가르시니아(체중 감량에 큰 영향 없다는 연구결과가 있음)

## Data visualization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
%config InlineBackend.figure_format = 'retina'

## 한글 글꼴 설정

In [3]:
# 한글 폰트 '나눔바른고딕' 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (3,179 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: fall

In [3]:
plt.rc('font', family='NanumBarunGothic')

## 데이터 불러오기

In [4]:
import os

# 다운로드 받은 파일을 판다스의 read_csv 를 통해 읽어옵니다.
# 파일을 읽어온 후 shape 로 행과 열의 수를 출력합니다.
csv_path = '/content/drive/MyDrive/health/health_data.csv'
df2 = pd.read_csv(csv_path, encoding='utf-8')

df = df2.copy()
pd.set_option('display.max_columns', None) # 컬럼 수가 많으므로 요약되어 표시되지 않게 합니다.
df # 데이터를 확인합니다.

,성별,나이,bmi,비만여부,혈색소,빈혈여부,식전혈당(공복혈당),당뇨여부,(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,음주여부,간 수치
0,0,40,22.0,정상,15.0,빈혈 아님,90.0,당뇨 없음,21.0,정상,27.0,정상,0.777778,21.0,0.777778,0.0,정상
1,1,60,28.9,비만,12.7,빈혈,87.0,당뇨 없음,18.0,정상,15.0,정상,1.200000,15.0,1.000000,0.0,정상
2,1,50,22.9,정상,12.8,빈혈,102.0,당뇨,27.0,정상,25.0,정상,1.080000,7.0,0.280000,0.0,정상
3,0,60,27.3,비만,16.4,빈혈 아님,146.0,당뇨,65.0,비정상,97.0,비정상,0.670103,72.0,0.742268,0.0,정상
4,1,50,20.8,정상,12.4,빈혈,110.0,당뇨,18.0,정상,17.0,정상,1.058824,14.0,0.823529,0.0,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987327,1,60,26.2,비만,13.6,빈혈 아님,124.0,당뇨,21.0,정상,25.0,정상,0.840000,18.0,0.720000,0.0,정상
987328,1,50,21.5,정상,14.0,빈혈 아님,98.0,당뇨 없음,21.0,정상,14.0,정상,1.500000,19.0,1.357143,1.0,정상
987329,0,40,22.9,정상,16.0,빈혈 아님,83.0,당뇨 없음,26.0,정상,20.0,정상,1.300000,17.0,0.850000,1.0,정상
987330,1,40,27.3,비만,11.9,빈혈,111.0,당뇨,25.0,정상,29.0,정상,0.862069,13.0,0.448276,0.0,정상


### 결측치 확인

In [6]:
# Null 값 여부 확인
df.isnull().sum()

성별            0
나이            0
bmi           0
비만여부          0
혈색소           0
빈혈여부          0
식전혈당(공복혈당)    0
당뇨여부          0
(혈청지오티)AST    0
AST구분         0
(혈청지오티)ALT    0
ALT구분         0
AST/ALT비율     0
감마 지티피        0
GTP/ALT비율     0
음주여부          0
간 수치          0
dtype: int64

In [7]:
obj_cols = df.select_dtypes('object').columns   # object 변수
obj_cols

Index(['비만여부', '빈혈여부', '당뇨여부', 'AST구분', 'ALT구분', '간 수치'], dtype='object')

In [8]:
flo_cols = df.select_dtypes('float64').columns   # float64 변수
flo_cols

Index(['bmi', '혈색소', '식전혈당(공복혈당)', '(혈청지오티)AST', '(혈청지오티)ALT', 'AST/ALT비율',
       '감마 지티피', 'GTP/ALT비율', '음주여부'],
      dtype='object')

In [9]:
int_cols = df.select_dtypes('int64').columns   # int64 변수
int_cols

Index(['성별', '나이'], dtype='object')

## 데이터 라벨링
- 비만 :  시서스 또는 가르시니아(체중 감량에 큰 영향 없다는 연구결과가 있음)
- 빈혈 : 철분제
- 당뇨 : metformine- 또는 오메가3
- 간 :  우루사

**BMI(비만여부)** + 시서스 or 가르시니아(체중 감량에 큰 영향 없다는 연구결과가 있음)

In [10]:
def class_bmi_div (bmi) :
    if bmi < 18.5 :  # 18.5 미만
        return "0" # 저체중
    elif 18.5 <= bmi < 23 :
        return "0" # 정상체중
    elif 23 <= bmi < 25 :
        return "1" # 과체중
    elif 25 <= bmi < 30 :  # 질환의심: 30 이상
        return "1" # 경도비만
    elif 30 <= bmi < 35 :
        return "1" # 중정도비만
    elif 35 <= bmi :
        return "1" # 고도비만

df['비만_여부']=df['bmi'].apply(class_bmi_div)

In [11]:
df.head()

,성별,나이,bmi,비만여부,혈색소,빈혈여부,식전혈당(공복혈당),당뇨여부,(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,음주여부,간 수치,비만_여부
0,0,40,22.0,정상,15.0,빈혈 아님,90.0,당뇨 없음,21.0,정상,27.0,정상,0.777778,21.0,0.777778,0.0,정상,0
1,1,60,28.9,비만,12.7,빈혈,87.0,당뇨 없음,18.0,정상,15.0,정상,1.200000,15.0,1.000000,0.0,정상,1
2,1,50,22.9,정상,12.8,빈혈,102.0,당뇨,27.0,정상,25.0,정상,1.080000,7.0,0.280000,0.0,정상,0
3,0,60,27.3,비만,16.4,빈혈 아님,146.0,당뇨,65.0,비정상,97.0,비정상,0.670103,72.0,0.742268,0.0,정상,1
4,1,50,20.8,정상,12.4,빈혈,110.0,당뇨,18.0,정상,17.0,정상,1.058824,14.0,0.823529,0.0,정상,0


---

**혈색소(빈혈여부)** + 철분제

In [12]:
cond1 = (df['성별'] == '1')
cond2 = (df['혈색소'] < 12)
cond3 = (df['혈색소'] < 13)

df.loc[cond1&cond2, '빈혈여부'] = "1" # 여자 and 혈색소 < 12 : 빈혈
df.loc[~cond1&~cond2, '빈혈여부'] = "0" # 여자 and 혈색소 >= 12 : 빈혈 아님
df.loc[~cond1&cond3, '빈혈여부'] = "1" # 남자 and 혈색소 < 13 : 빈혈
df.loc[~cond1&~cond3, '빈혈여부'] = "0" # 남자 and 혈색소 >= 13 : 빈혈 아님 

df['빈혈_여부']=df['빈혈여부'].astype('object')

In [13]:
df.head()

,성별,나이,bmi,비만여부,혈색소,빈혈여부,식전혈당(공복혈당),당뇨여부,(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,음주여부,간 수치,비만_여부,빈혈_여부
0,0,40,22.0,정상,15.0,0,90.0,당뇨 없음,21.0,정상,27.0,정상,0.777778,21.0,0.777778,0.0,정상,0,0
1,1,60,28.9,비만,12.7,1,87.0,당뇨 없음,18.0,정상,15.0,정상,1.200000,15.0,1.000000,0.0,정상,1,1
2,1,50,22.9,정상,12.8,1,102.0,당뇨,27.0,정상,25.0,정상,1.080000,7.0,0.280000,0.0,정상,0,1
3,0,60,27.3,비만,16.4,0,146.0,당뇨,65.0,비정상,97.0,비정상,0.670103,72.0,0.742268,0.0,정상,1,0
4,1,50,20.8,정상,12.4,1,110.0,당뇨,18.0,정상,17.0,정상,1.058824,14.0,0.823529,0.0,정상,0,1


---

**식전혈당(공복혈당) 당뇨여부** + metformine- or 오메가3


In [14]:
def trans(tr) :
    if 70 <= tr < 100 :
        return "0" # 정상
    elif 100 <= tr < 126 :
        return "1" # 비정상(공복혈당장애)
                 # 100 ~ 110 : 100 미만 대비 당뇨병 위험도가 3.17배, 110 ~ 126 : 100 미만 대비 당뇨병 위험도가 7.42배
    else :
        return "1" # 당뇨병
        # 정상(저혈당, 정상) / 비정상(공복혈당장애, 당뇨병)

df['당뇨_여부']=df['식전혈당(공복혈당)'].apply(trans).astype('object')

In [15]:
df.head()

,성별,나이,bmi,비만여부,혈색소,빈혈여부,식전혈당(공복혈당),당뇨여부,(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,음주여부,간 수치,비만_여부,빈혈_여부,당뇨_여부
0,0,40,22.0,정상,15.0,0,90.0,당뇨 없음,21.0,정상,27.0,정상,0.777778,21.0,0.777778,0.0,정상,0,0,0
1,1,60,28.9,비만,12.7,1,87.0,당뇨 없음,18.0,정상,15.0,정상,1.200000,15.0,1.000000,0.0,정상,1,1,0
2,1,50,22.9,정상,12.8,1,102.0,당뇨,27.0,정상,25.0,정상,1.080000,7.0,0.280000,0.0,정상,0,1,1
3,0,60,27.3,비만,16.4,0,146.0,당뇨,65.0,비정상,97.0,비정상,0.670103,72.0,0.742268,0.0,정상,1,0,1
4,1,50,20.8,정상,12.4,1,110.0,당뇨,18.0,정상,17.0,정상,1.058824,14.0,0.823529,0.0,정상,0,1,1


---

**AST/ALT비율, GTP/ALT비율(간 수치)** + 우루사

**AST/ALT 비율**

In [16]:
df['AST/ALT비율'] = df['(혈청지오티)AST']/df['(혈청지오티)ALT'] # AST/ALT비율

**감마지티피/ALT비율**

In [17]:
df['GTP/ALT비율'] = df['감마 지티피']/df['(혈청지오티)ALT']

**알코올성 간질환 여부**

In [18]:
# AST/ALT비율 > 1
# 감마지티피/ALT > 2.5
# AST는 300을 넘지 않음
# ALT는 종종 정상
# AST/ALT비율이 2~3 이상인 경우 알코올성 간질환을 강하게 시사한다.

cond1 = ((df['AST/ALT비율']>1)&(df['GTP/ALT비율']>2.5)&(df['(혈청지오티)AST']<=300))

df.loc[cond1, '간_수치'] = '1'
df.loc[~cond1, '간_수치'] = '0'

In [19]:
df.tail()

,성별,나이,bmi,비만여부,혈색소,빈혈여부,식전혈당(공복혈당),당뇨여부,(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,음주여부,간 수치,비만_여부,빈혈_여부,당뇨_여부,간_수치
987327,1,60,26.2,비만,13.6,0,124.0,당뇨,21.0,정상,25.0,정상,0.840000,18.0,0.720000,0.0,정상,1,0,1,0
987328,1,50,21.5,정상,14.0,0,98.0,당뇨 없음,21.0,정상,14.0,정상,1.500000,19.0,1.357143,1.0,정상,0,0,0,0
987329,0,40,22.9,정상,16.0,0,83.0,당뇨 없음,26.0,정상,20.0,정상,1.300000,17.0,0.850000,1.0,정상,0,0,0,0
987330,1,40,27.3,비만,11.9,1,111.0,당뇨,25.0,정상,29.0,정상,0.862069,13.0,0.448276,0.0,정상,1,1,1,0
987331,0,50,21.5,정상,14.9,0,121.0,당뇨,17.0,정상,13.0,정상,1.307692,24.0,1.846154,0.0,정상,0,0,1,0


In [20]:
df.drop(['비만여부','당뇨여부','간 수치','음주여부'], axis=1, inplace=True)
df.head()

,성별,나이,bmi,혈색소,빈혈여부,식전혈당(공복혈당),(혈청지오티)AST,AST구분,(혈청지오티)ALT,ALT구분,AST/ALT비율,감마 지티피,GTP/ALT비율,비만_여부,빈혈_여부,당뇨_여부,간_수치
0,0,40,22.0,15.0,0,90.0,21.0,정상,27.0,정상,0.777778,21.0,0.777778,0,0,0,0
1,1,60,28.9,12.7,1,87.0,18.0,정상,15.0,정상,1.200000,15.0,1.000000,1,1,0,0
2,1,50,22.9,12.8,1,102.0,27.0,정상,25.0,정상,1.080000,7.0,0.280000,0,1,1,0
3,0,60,27.3,16.4,0,146.0,65.0,비정상,97.0,비정상,0.670103,72.0,0.742268,1,0,1,0
4,1,50,20.8,12.4,1,110.0,18.0,정상,17.0,정상,1.058824,14.0,0.823529,0,1,1,0


In [21]:
df.drop(['ALT구분','AST구분','AST/ALT비율','GTP/ALT비율','(혈청지오티)AST','(혈청지오티)ALT','감마 지티피'], axis=1, inplace=True)
df.head()

,성별,나이,bmi,혈색소,빈혈여부,식전혈당(공복혈당),비만_여부,빈혈_여부,당뇨_여부,간_수치
0,0,40,22.0,15.0,0,90.0,0,0,0,0
1,1,60,28.9,12.7,1,87.0,1,1,0,0
2,1,50,22.9,12.8,1,102.0,0,1,1,0
3,0,60,27.3,16.4,0,146.0,1,0,1,0
4,1,50,20.8,12.4,1,110.0,0,1,1,0


In [22]:
df.drop(['bmi','혈색소','빈혈여부','식전혈당(공복혈당)'], axis=1, inplace=True)
df.head()

,성별,나이,비만_여부,빈혈_여부,당뇨_여부,간_수치
0,0,40,0,0,0,0
1,1,60,1,1,0,0
2,1,50,0,1,1,0
3,0,60,1,0,1,0
4,1,50,0,1,1,0


---

In [23]:
df.to_csv('/content/drive/MyDrive/health_data3.csv',index=False)